In [2]:
#Import libraries 
import numpy as np
import math
import matplotlib.pyplot as plt


In [3]:
#Define constants 

pi = np.pi
deg2rad = pi/180
rad2deg = 1/deg2rad


# This is the main script in the steady state version of the North model

# Set physical parameters
S0 = 1354.0 #The solar constant, corresponds to Q_0=338.5 W/m2
L = 1 #the relative strength of the solar constant, 1 = present
A = 203.3 #Infrared flux A-constant. In units W/m2. In North A = 201.4
B = 2.09 #Infrared flux B-constant. In units W/(m2 * K). In North B = 1.45 
Dmag = 0.44 #Infrared flux D-constant. (Absorps B) In units W/(m2 * K). 
Cl = 9.8 #unit W/m2 * yr/K. Heat Capacity of the system
Toffset = 0 
coldstart = 0 #1, then Toffset is set to -40 for a cold initial temperature
hadleyflag = 0 #0: no hadley cell
albedoflag = 1 #1: albedo feedback with T_crit, otherwise a fixed boundary at 72deg

# Define a function that initializes the model and calculates T
# Set model parameters
jmx = 251 #antallet af dataframes i some form
delt = 1/50 
NMAX = 100000
delx = 2/jmx #bredden på x?

# define an array with latitude steps in both x and phi
x = np.linspace(-1, 1, jmx)
phi = np.arcsin(x) * rad2deg




In [4]:
# Define the latitude dependent insolation function. xs is the latitudal variable
def Q_lat(xs, L=1 , S0=1354.0):    
    return  L * (S0/4) * (1 - 0.241 * (3 * np.square(xs) - 1))

In [5]:
# Define the albedo function
def albedo(T, jmx, albedoflag):
    alb = np.ones(jmx) * 0.3 #this is the general albedo value for the Earth on average
    if (albedoflag == 1):
        k = np.argwhere(T <= -10) 
        alb[k] = 0.6# recalculate albedo for snowline at T=-10C. If the temperature is below -10, we have ice albedo of 0.6
    else:
        k = np.argwhere(abs(x) >= 0.95) #we have a ice cap at each pole
        alb [k] = 0.6# recalculate albedo for fixed snowline
    return alb

In [7]:
# Define the heat diffusion function
def D_lat(xs, D0=0.44, hadleyflag=False):
    if hadleyflag == 1:
        return D0 *(1 + 9 * np.exp(-(xs/np.sin(30 * np.pi/180))**6)) #latitude dependent diffusion with hadley cell
    else:
        return D0 * np.ones(len(xs)) # constant diffusion without hadley cell

In [9]:
# Define functions used for matrix operations when solving the equations

# Define the intermediate function to calculate matrix coefficients
def setupfastMh(delx, jmx, D, B, Cl, delt):
    #set up lambda array.
    lam = (1 - np.arange(-1, 1, delx) * np.arange(-1 ,1 , delx))/(delx * delx)
    lam = D[0:jmx] * lam
    
    M = np.zeros((jmx, jmx))
    
    # do something special at the boundaries
    M[0, 0] = -B - lam[1] #first points
    M[0, 1] = lam[1]
    M[jmx-1, jmx-2] = lam[jmx - 1] # last points
    M[jmx-1, jmx-1] = -B - lam[jmx - 1]
    for jj in range(jmx - 2):
        j = jj + 2
        M[j - 1, j - 2] = lam[j - 1]
        M[j - 1, j - 1] = -B - (lam[j] + lam[j - 1])
        M[j - 1, j] = lam[j]
    # add in heat capacities
    M = M / Cl
    # calculate the inverse of M', the matrix operator.
    Mh= M
    return Mh

def setupfastinvM(Mh, jmx, delt):
    M = 0.5 * Mh
    for j in range(jmx):
        M[j, j]=M[j, j] - 1/delt
    invM = np.linalg.inv(M) 
    return invM


In [10]:
# Calculate an array of annual mean insolation for the latitudes steps
S = Q_lat(x, L, S0)  
    
# Calculate an array of diffussion constants
xmp = np.arange(-1, 1 + delx, delx)
D = D_lat(xmp, Dmag, hadleyflag)

# define initial T array
T = 0 * (1 - 2 * np.square(x))
if coldstart == 1:
    Toffset = -40
T = T + Toffset
Tinit = T

# Calculate global mean temperature
Tglob = np.mean(T) #ok to take the mean when we use x

# Calculate initial albedo
alb = albedo(T, jmx, albedoflag)

# Initial boundary conditions
src = (1 - alb) * S/Cl -A/Cl
Mh = setupfastMh(delx, jmx, D, B, Cl, delt)
invM = setupfastinvM(Mh, jmx, delt)
h = np.dot(Mh, T) + src


In [11]:
# Start time-stepping loop
for n in range(2): #Should be NMAX
    Tglob_prev = Tglob
    # calculate src for this loop
    alb = albedo(T, jmx, albedoflag)
    src = (1 - alb) * S/Cl - A/Cl
    # Calculate new T.
    T = -np.dot(invM, 0.5* (h + src) + T/delt)
    
    # Calculate h for next loop.
    h = np.dot(Mh, T) + src
    # Check to see if global mean temperature has converged
    Tglob = np.mean(T)
    #print(Tglob)
    Tchange = Tglob - Tglob_prev
    if abs(Tchange)< 1e-12:
        break



In [12]:
# Compute meridional heat flux and its convergence
a = 6.37e+6; # earth radius in meters
Mh = setupfastMh(delx,jmx,D,0,1,delt)
invM = setupfastinvM(Mh,jmx,delt)
Dmp = D[0:jmx]
divF = np.dot(Mh,T)
F = -2 * math.pi * a * a * np.sqrt(1 - x*x) * Dmp * np.gradient(T,delx)
